# 
We are using a dataset on beer purchases. The goal is to predict if light beer purchased in the US is BUD light. To achieve this goal, we will use the information provided by the following socioeconomic characteristics:
* market           - where the beer is bought
* buyertype        - who is the buyer () 
* income           - ranges of income
* childrenUnder6   - does the buyer have children under 6 years old
* children6to17    - does the buyer have children between 6 and 17
* age              - bracketed age groups
* employment       - fully employed, partially employed, no employment.
* degree           - level of occupation
* occuptation      - which sector you are employed in
* ethnic           - white, asian, hispanic, black or other
* microwave        - own a microwave
* dishwasher       - own a dishwasher
* tvcable          - what type cable tv subscription you have
* singlefamilyhome - are you in a single family home
* npeople          - number of people you live with 1,2,3,4, +5

First, we load the dataset and create an output variable that indicates purchases of Bud Light.

In [5]:
import pandas as pd
import numpy as np
import os
 
file_path = r"C:\Users\jamak\OneDrive\Lund\Machine Learning\LightBeer2.csv"
lb    = pd.read_csv(file_path)
y     = np.zeros(shape=lb.shape[0])
y[lb['beer_brand'] == "BUD LIGHT"]     = 1
demog = lb.iloc[:,9:]
demog = pd.get_dummies(demog, drop_first=True)



We also split the data into training and test sets:

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(demog, y, train_size=0.75, shuffle=False)

stdz_X = StandardScaler().fit(X_train)

X_train = stdz_X.transform(X_train)
X_test  = stdz_X.transform(X_test)


## Part 1: Specifying and training neural networks
We will now start building a neural network to predict the purchase of Bud Light.

We start with specifying the architecture of our very first and very small neural net `model1`.
Add three layers to `model1`, two hidden layers with $30$ and $15$ hidden units, respectively, and an output layer.
For the two hidden layers you should use the ReLU activation function. Additionally, We will choose an approriate activation function for the classification problem.

In [7]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

# Initialize first moodel
model1 = keras.Sequential()

# add input layer
model1.add(Input(shape=(X_train.shape[1],)))

# Add first hidden layer with 30 hidden units
model1.add(Dense(30, activation='relu'))

# Add second hidden layer with 15 hidden units
model1.add(Dense(15, activation='relu'))

# Add output layer Binary classification thus sigmoid as activation function
model1.add(Dense(1, activation='sigmoid'))




Next, we compile our model specification. We will choose a suitable loss function for our classification problem. As optimization algorithm we wil use *Adam* with learning rate $0.00003$. Lastly, use `accuracy` as a metric.

In [8]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

# Model with binarycrossentropy as loss Adam as optimzer and learning rate of 00003
model1.compile(optimizer= Adam(learning_rate=0.00003),
               loss=BinaryCrossentropy(),
               metrics=['accuracy'])

# Training of Model 
Now we wil train the model using $250$ epochs, a batch_size of $2^8$, and use $25\%$ of the data for validation.

In [9]:
# Training model
train = model1.fit(X_train, y_train,
                     epochs=250,
                     batch_size=2**8,
                     validation_split= 0.25)

loss_valloss_difference_1c = ("We first see that the validation loss is decreasing, which is a clear indication that the model is effectivly learning"
                              "As the learning continues zig-zag patterns arise (increases/decreases in val loss) even with adam's method patterns like this "
                              "Here we probably run into saddle points, plateaus or local minima's. Adam is specifically designed to migate but can not entirely eliminate"
                              "After around epoch 150 we see a gradually increase in the val error which indicates our model starts to overfit. We could implement" 
                              "methods like stopping rule to migtate this problem")

Epoch 1/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4460 - loss: 0.8321 - val_accuracy: 0.4217 - val_loss: 0.8089
Epoch 2/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4848 - loss: 0.7767 - val_accuracy: 0.5241 - val_loss: 0.7122
Epoch 3/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5185 - loss: 0.7458 - val_accuracy: 0.6008 - val_loss: 0.6463
Epoch 4/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5344 - loss: 0.7194 - val_accuracy: 0.6701 - val_loss: 0.6049
Epoch 5/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5485 - loss: 0.7046 - val_accuracy: 0.7180 - val_loss: 0.5781
Epoch 6/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5604 - loss: 0.6913 - val_accuracy: 0.7650 - val_loss: 0.5597
Epoch 7/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5667 - loss: 0.6807 - val_accuracy: 0.7932 - val_loss: 0.5498
Epoch 8/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5758 - loss: 0.6759 - val_accu

### 
Early stopping can be used to avoid overfitting. We will apply this here, with `patience` argument set to 20.  `Model1b`, which otherwise should have a setup identical to `model1` as a copy of it.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define architecture here
model1b = keras.Sequential()

model1b.add(Input(shape=(X_train.shape[1],)))

model1b.add(Dense(30, activation='relu'))

model1b.add(Dense(15, activation='relu'))


model1b.add(Dense(1, activation='sigmoid'))


model1b.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#  early stopping monitor
early_stopping_monitor = EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True,
    verbose=1
)


model1b_fit = model1b_fit = model1b.fit(
    X_train, y_train,
    validation_split=0.25,
    epochs=250,
    batch_size=256,
    callbacks=[early_stopping_monitor]
)

stopped_epoch = len(model1b_fit.history['loss'])

when_earlystop_1d = f"The training stopped after {stopped_epoch} epochs."

Epoch 1/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5743 - loss: 0.6815 - val_accuracy: 0.7982 - val_loss: 0.5413
Epoch 2/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6878 - loss: 0.5845 - val_accuracy: 0.7549 - val_loss: 0.5483
Epoch 3/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7300 - loss: 0.5435 - val_accuracy: 0.7246 - val_loss: 0.5744
Epoch 4/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7590 - loss: 0.5079 - val_accuracy: 0.7008 - val_loss: 0.5844
Epoch 5/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7722 - loss: 0.4842 - val_accuracy: 0.6833 - val_loss: 0.6231
Epoch 6/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7922 - loss: 0.4589 - val_accuracy: 0.6528 - val_loss: 0.6479
Epoch 7/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8054 - loss: 0.4391 - val_accuracy: 0.6514 - val_loss: 0.6694
Epoch 8/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8156 - loss: 0.4244 - val_accu

In [11]:
res_model1 = model1b.evaluate(X_test, y_test)
print(res_model1)
difference_in_accuracy_1e = "The test accuracy is approx. 0.71 and validation accuracy is 0.62 so the difference is 0.09"


572/572 ━━━━━━━━━━━━━━━━━━━━ 0s 766us/step - accuracy: 0.5320 - loss: 0.7037
[0.5843103528022766, 0.7067673206329346]


# Model Evaluation 
Now we will use the `confusion_matrix()` function from the `metrics` module of scikit-learn to disaggregate model performance to class-specific performance. 

In [ ]:
from sklearn.metrics import confusion_matrix

prob_model1 = model1b.predict(X_test)
CM_model1   = confusion_matrix(y_test, prob_model1 > 0.5)

TN = CM_model1[0, 0]
TP = CM_model1[0, 1]
FN = CM_model1[1, 0]
FP = CM_model1[1, 1]


TPR_1f = TP / (TP + FN)
FPR_1f = FP / (FP + TN)

print(CM_model1)
print(TPR_1f)
print(FPR_1f)



572/572 ━━━━━━━━━━━━━━━━━━━━ 1s 880us/step
[[11064  1817]
 [ 3543  1855]]
0.3389925373134328
0.14358696493536652



"The model's performance varies significantly between classes. The True Positive Rate (TPR) of 38.2%"
                            "suggests moderate effectiveness in identifying positive cases, as it correctly identifies about 38.2% of actual positives."
                            "However, the False Positive Rate (FPR) of 16.2% indicates that the model incorrectly labels 16.2% of negatives as positives,"
                            "which might be problematic in scenarios where false positives are costly. These results suggest that the model's accuracy is"
                            "not approximately equal across categories, with a specific weakness in distinguishing negatives without falsely labeling them as positives."

Here we will use $\ell_2$ regularization to update the weights. `Model2` which is identical to that of `model1b` except for $\ell_2$ regularization with regularization factor `l2_pen` in the two hidden layers.  

Then we compile and fit this regularized model with the same parameters as previous method.

In [ ]:
from tensorflow.keras.regularizers import l2
l2_pen = 0.005

# 1.

model2 = keras.Sequential()

model2.add(Input(shape=(X_train.shape[1],)))

model2.add(Dense(30, activation='relu', kernel_regularizer=l2(l2_pen)))

model2.add(Dense(15, activation='relu', kernel_regularizer=l2(l2_pen)))

model2.add(Dense(1, activation='sigmoid'))

model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# early stopping callback
early_stopping_monitor = EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True,
    verbose=1
)

# 2
model2_fit = model2.fit(
    X_train, y_train,
    validation_split=0.25,
    epochs=250,
    batch_size=256,
    callbacks=[early_stopping_monitor]
)


Epoch 1/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6042 - loss: 0.9196 - val_accuracy: 0.8831 - val_loss: 0.6522
Epoch 2/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6815 - loss: 0.7019 - val_accuracy: 0.7876 - val_loss: 0.5945
Epoch 3/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7154 - loss: 0.6289 - val_accuracy: 0.6831 - val_loss: 0.6510
Epoch 4/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7467 - loss: 0.5845 - val_accuracy: 0.6828 - val_loss: 0.6466
Epoch 5/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7677 - loss: 0.5527 - val_accuracy: 0.6480 - val_loss: 0.6528
Epoch 6/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7873 - loss: 0.5273 - val_accuracy: 0.6351 - val_loss: 0.7184
Epoch 7/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7967 - loss: 0.5098 - val_accuracy: 0.6055 - val_loss: 0.7304
Epoch 8/250
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8037 - loss: 0.4956 - val_accu


We compared the prediction accuracy on test and training sets. However, this is bad measure when the data is not well balanced (in terms of the observed output categories). Instead, one can use the cross entropy for the binomial distribution (minus the average log likelihood of the model). In fact, we chose this function as loss function for model training when we compiled `model1` and `model2`.

To compare the test error of `model2` to that of `model1b` we don't want to use `loss` from `evaluate` since this includes the $\ell_2$ penalty.  In the library `MLmetrics` the function `log_loss()` computes the cross entropy for the binomial distribution without penalty term. 

We will get predicted output probabilities on test data from `model2` and save them as `prob_model2`.
Then we use `log_loss()` from the metrics module in scikit-learn to compute the cross-entropy loss for `model2` on the test data and save it as `logloss_model2`. 
Then we describe how the accuracy on test data differs between `model1b` and `model2`.

In [ ]:
from sklearn.metrics import log_loss

# 1. 
prob_model2    = model2.predict(X_test)

# 2.
logloss_model2 = log_loss(y_test, prob_model2)
print(logloss_model2)

# 3.



572/572 ━━━━━━━━━━━━━━━━━━━━ 1s 806us/step
0.5614786502640219


"The loss for the model including regularization(model2) is approx 0.54 which is better than the 0.59 of model 1b."
                            "This indicates model 2 is performing better in terms of genaralization"

## Part 2: Tuning neural nets with caret

Keras provides functions that allow the use of scikit-learn for model tuning. Using this functionality requires relatively little effort and in this part we are going to practice the individual steps of model tuning with `sklearn`.


First, we need to define the architecture of the neural net that we tune and to compile the model. This needs to be done inside a function. The arguments of this function are the tuning parameters whose candidate values we want to feed into the function one by one.

We will work with the architecture defined for `model2`. The only parameter that we want to tune is the regularization parameter for $\ell_2$ penalization inside the hidden units.
Now we create a function `modelbuild_2a` which has one argument `l2_pen`. In this function, specify the architecture of a Keras model `model`, identical to that of `model2` and with $\ell_2$-penalty set to `l2_pen`. We Compile the model with the same settings and return it as function output.

In [15]:
# Create the function to build the model with L2 regularization
def modelbuild_2a(l2_pen):
    model2 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(30, activation='relu', kernel_regularizer=l2(l2_pen)),
        Dense(15, activation='relu', kernel_regularizer=l2(l2_pen)),
        Dense(1, activation='sigmoid')
    ])
    model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model2


early_stopping_monitor = EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True,
    verbose=1
)


In order to make the function `modelbuild_2a` compatible with `sklearn`, we need to call it inside the `KerasClassifier()` function from the `wrappers` module of `scikit-learn`. I created a class due too problems with the module KerasCalssifier

In [16]:

from scikeras.wrappers import KerasClassifier

""" Grid Search Cv kept cycling through all hyperparameter combinations in  a cycle disregarding the early stop,
 so I created a VerboseKeras Classifier to limit this.

"""
class VerboseKerasClassifier(KerasClassifier):
    def fit(self, X, y, **kwargs):
        print(f"Training model with l2_pen: {self.l2_pen}")
        return super().fit(X, y, **kwargs)

model2_sklearn_spec = VerboseKerasClassifier(
    model=modelbuild_2a,
    l2_pen=0.005,
    epochs=250,
    batch_size=256,
    validation_split=0.25,
    verbose=0,
    callbacks=[early_stopping_monitor]
)

Next, we define a parameter grid `tune_grid_2c`. This must be a dictionary object.Its values is zero as well as $10^{r}$ for a grid of eleven $r$-values from $-4$ and $-1$ at equal distance.


In [17]:
# grid for tuning l2_pen
# Create L2 penalty values
r_values = np.linspace(-4, -1, 11)
l2_pen_values = np.concatenate(([0], 10**r_values))
tune_grid_2c = {
    'l2_pen': l2_pen_values[:3]
}


Now, we can tune our model. That's computationally quite costly, so we will use merely a fraction of the available training data. 

In [18]:
X_train_small,_ , y_train_small, _ = train_test_split(X_train, y_train, train_size=0.3, random_state=6)

we will do the following:

1. Use `Kfold()` from the model selection module in scikit-learn to define a random partition of the training data into five folds. and use $5$ as your random seed. 
2. Call `GridSearchCV()` and use the wrapper function from before, the parameter grid as well as `cv_splits_2d` as arguments.
3. Apply the `fit()`-method to `GridSearchCV` and use `X_train_small` and `y_train_small` as inputs and outputs.

In [19]:
from sklearn.model_selection import (KFold, GridSearchCV)
cv_splits_2d = KFold(n_splits=5, shuffle=True, random_state=5)
import time

#1 CV KFold 
cv_splits_2d = KFold(n_splits=5, shuffle=True, random_state=5)

# 2 GridSearchCV setup
NN_tune_2d = GridSearchCV(
    estimator=model2_sklearn_spec,
    param_grid=tune_grid_2c,
    cv=cv_splits_2d,
    verbose=2,
    n_jobs=1
)

# 3 Fit the model
start_time = time.time()
nn_trainlog = NN_tune_2d.fit(X_train_small, y_train_small)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Cross validation finished in {elapsed_time:} seconds")
print("Best parameters:", NN_tune_2d.best_params_)
print("Best score:", NN_tune_2d.best_score_)



Fitting 5 folds for each of 3 candidates, totalling 15 fits
Training model with l2_pen: 0.0
Epoch 51: early stopping
Restoring model weights from the end of the best epoch: 31.
[CV] END .........................................l2_pen=0.0; total time=   6.7s
Training model with l2_pen: 0.0
Epoch 67: early stopping
Restoring model weights from the end of the best epoch: 47.
[CV] END .........................................l2_pen=0.0; total time=   8.2s
Training model with l2_pen: 0.0
Epoch 57: early stopping
Restoring model weights from the end of the best epoch: 37.
[CV] END .........................................l2_pen=0.0; total time=   7.1s
Training model with l2_pen: 0.0
Epoch 52: early stopping
Restoring model weights from the end of the best epoch: 32.
[CV] END .........................................l2_pen=0.0; total time=   6.6s
Training model with l2_pen: 0.0
Epoch 58: early stopping
Restoring model weights from the end of the best epoch: 38.
[CV] END ......................

By default, GridSearchCV saves the trained model with the best tuning parameter values as as object `best_estimator_` inside `NN_tune_2d`. However, in our case this model is a KerasClassifier object. We cannot use such an object for making predictions on test data. 
We will extract this object-inside-the-object-inside-the-object and save it as `model3`.

In [20]:
model3 = NN_tune_2d.best_estimator_.model


## Part 3: Saving, loading and retraining neural nets 

Training and tuning neural nets can take a lot of time. Therefore, it is possible to save entire fitted models to disk and to import them at a later point in time. Apply the `save()`-method to your most recent `model3` in order to save it 

we will use the file explorer in your operating system to look how exactly the model was saved on my hard drive. 

In [25]:
# Define the path 
save_path = os.path.join(os.getcwd(), 'DABN13_asst6_saved_model3.keras')

# Save the model
model3.save(save_path)

# Save the model to new name 
saved_model_3a = save_path

AttributeError: 'function' object has no attribute 'save'

Now, we use the `load_model` function in the `models` module of Keras to load your saved model into your python session again. Save this model as `model4`.


In [ ]:
from tensorflow.keras.models import load_model
#Load saved model
model4 = load_model('DABN13_asst6_saved_model3.keras')


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# set early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Retrain the model
history = model4.fit(
    X_train, y_train,
    epochs=250,
    batch_size=2**8,
    validation_split=0.1,
    callbacks=[early_stopping]
)

# Get predicted probabilities
prob_model4 = model4.predict(X_test)

# Calculate log loss
from sklearn.metrics import log_loss
logloss_model4 = log_loss(y_test, prob_model4)

# Calculate accuracy
accuracy_model4 = model4.evaluate(X_test, y_test)[1]

performance_comparison_3c = f"""
The log loss of model4 on the test set is {logloss_model4:.6f}.
The accuracy of model4 on the test set is {accuracy_model4:.6f}.

Compared to model2 (which had an accuracy of {res_model1[1]:.6f}), 
model4s accuracy was higher at {accuracy_model4} comapred to {res_model1[1]:.6f}. 
The model improved by {abs(accuracy_model4 -res_model1[1]):.5f}. 

This performance improvement was due to the hyperparameter tuning and training
on the full dataset. The hyperparameter tuning helped find a more optimal model architecture.
"""
print(performance_comparison_3c)

## Part 4: Manual predictions from a trained neural net

In this part we will build predictions *manually* by extracting weights from the trained  `model2` and by constructing the transformations in the layers of the neural net ourselves.

creating our own ReLU activation function. Then, write our own sigmoid function for the output layer transformation. 

In [ ]:
# ReLU activation function
def ReLU(x):
    return np.maximum(0, x)

# Sigmoid activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

Now we are going to use the equations both hidden units and output unit to construct output predictions for the $n_{test}$ data points in `X_test`. WE WILL DO THE FOLLOWING

1. Apply the `get_weights()` method on `model2` to obtain a list object which stores the weights and biases of the learned model. 
2. Extract the objects inside `weight_and_bias_4b` into the objects for $\mathbf{b}_1,\mathbf{W}_1, \mathbf{b}_2, \mathbf{W}_2,\mathbf{b}_3, \mathbf{W}_3$ defined in the code chunk below.
3. Construct the linear term of the first layer hidden units and THEN save these linear terms as a $30 \times n_{train}$ vector `Z_1`.
4. Construct the $15 \times n_{train}$ vector `Z_2` of linear terms for the second layer hidden units. Then, we obtain the linear term of the output unit and save it as `Z_3`.
5. Put `Z_3` into the output layer activation function in order to get predictions for the probability of a Bud Light purchase. Save this as $n_{train} \times 1$ vector `pred_own_2b`. 



In [ ]:
# 1. Extract weights and biases from the model 2
weight_and_bias_4b = model2.get_weights()

# 2. Extract weight and biases
W1 = weight_and_bias_4b[0]  
b1 = weight_and_bias_4b[1]  
W2 = weight_and_bias_4b[2]  
b2 = weight_and_bias_4b[3]  
W3 = weight_and_bias_4b[4]  
b3 = weight_and_bias_4b[5]  

# 3. Linear term for the first hidden layer
Z_1 = np.dot(X_test, W1) + b1
A_1 = ReLU(Z_1) # Apply written Relu function

# 4. Linear term for the second hidden layer
Z_2 = np.dot(A_1, W2) + b2
A_2 = ReLU(Z_2) 
#5 linear term for the output layer
Z_3 = np.dot(A_2, W3) + b3

# Apply sigmoid activation to Z_3
prob_model2_own_4b = prob_model2_own_4b = sigmoid(Z_3)